<a href="https://colab.research.google.com/github/AbdelazizAkl/TextClassificaiton/blob/main/TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.4 MB/s eta 0:00:00


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import string
import numpy as np

from datasets import load_dataset, Features, Value

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Load SST dataset
train_dataset = load_dataset("sst", split="train")
valid_dataset = load_dataset("sst", split="validation")
test_dataset = load_dataset("sst", split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for sst contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sst
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/8544 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1101 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2210 [00:00<?, ? examples/s]

In [ ]:
lemmatizer = WordNetLemmatizer()

# Preprocessing and lemmatization function
def preprocess_and_lemmatize(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation marks using regex
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize the tokens with POS tagging
    lemmatized_tokens = [lemmatizer.lemmatize(word, pos=get_wordnet_pos(tag)) for word, tag in nltk.pos_tag(filtered_tokens)]
    # Join the lemmatized tokens back into a string
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text

# Function to map POS tag to WordNet POS tag
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return 'a'  # Adjective
    elif tag.startswith('V'):
        return 'v'  # Verb
    elif tag.startswith('N'):
        return 'n'  # Noun
    elif tag.startswith('R'):
        return 'r'  # Adverb
    else:
        return 'n'  # Default to noun

# Apply preprocessing and lemmatization to each dataset split
train_dataset = train_dataset.map(lambda example: {'sentence': preprocess_and_lemmatize(example['sentence']), 'label': example['label']})
valid_dataset = valid_dataset.map(lambda example: {'sentence': preprocess_and_lemmatize(example['sentence']), 'label': example['label']})
test_dataset = test_dataset.map(lambda example: {'sentence': preprocess_and_lemmatize(example['sentence']), 'label': example['label']})

Parameter 'function'=<function <lambda> at 0x79b3c2bbb5b0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/8544 [00:00<?, ? examples/s]

Map:   0%|          | 0/1101 [00:00<?, ? examples/s]

Map:   0%|          | 0/2210 [00:00<?, ? examples/s]

In [ ]:
def process_dataset(dataset):
    # Define the mapping function for sentiment scores to integer labels
    def map_sentiment_score_to_int(example):
        score = example['label']
        if score <= 0.2:
            return {'label': 0}  # Very Negative
        elif score <= 0.4:
            return {'label': 1}  # Negative
        elif score <= 0.6:
            return {'label': 2}  # Neutral
        elif score <= 0.8:
            return {'label': 3}  # Positive
        else:
            return {'label': 4}  # Very Positive

    # Apply the mapping to convert labels
    mapped_dataset = dataset.map(map_sentiment_score_to_int)

    # Remove unwanted columns
    processed_dataset = mapped_dataset.remove_columns(['tokens', 'tree'])

    # Define new features with 'label' as int8
    new_features = Features({
        'sentence': Value('string'),
        'label': Value('int8')
    })

    # Cast the dataset to the new features schema
    final_dataset = processed_dataset.cast(new_features)

    return final_dataset

In [ ]:
# Process the train_dataset
train_dataset = process_dataset(train_dataset)

# Process the validation_dataset
valid_dataset = process_dataset(valid_dataset)

# Process the test_dataset
test_dataset = process_dataset(test_dataset)

y_train = np.array([example['label'] for example in train_dataset])
y_valid = np.array([example['label'] for example in valid_dataset])
y_test = np.array([example['label'] for example in test_dataset])


Map:   0%|          | 0/8544 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/8544 [00:00<?, ? examples/s]

Map:   0%|          | 0/1101 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1101 [00:00<?, ? examples/s]

Map:   0%|          | 0/2210 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/2210 [00:00<?, ? examples/s]

In [ ]:
# Print a few examples to verify modifications
print(train_dataset)
print(type(train_dataset[0]['label']))
print(train_dataset[:5]['label'])
print(valid_dataset[:5]['label'])
print(test_dataset[:5]['label'])
print(train_dataset[:5]['sentence'])
print(valid_dataset[:5]['sentence'])
print(test_dataset[:5]['sentence'])

Dataset({
    features: ['sentence', 'label'],
    num_rows: 8544
})
<class 'int'>
[3, 4, 3, 2, 3]
[3, 2, 3, 4, 4]
[2, 3, 4, 2, 4]
['rock destine 21st century new conan go make splash even great arnold schwarzenegger jeanclaud van damme steven segal', 'gorgeously elaborate continuation lord ring trilogy huge column word adequately describe cowriterdirector peter jackson expand vision jrr tolkien middleearth', 'singercomposer bryan adam contribute slew song potential hit simply intrusive story whole package certainly capture intend er spirit piece', 'think america would enough plucky british eccentric heart gold', 'yet act still charm']
['lovely film lovely performance buy accorsi', 'one go unindicted probably best', 'nearly move tears couple scene get ice water vein', 'warm funny engaging film', 'us sharp humor insight human nature examine class conflict adolescent yearn root friendship sexual identity']
['effective tootepid biopic', 'sometimes like go movie fun wasabi good place start

In [ ]:
def train_naive_bayes(D, C):
    # Create the vocabulary: set of all unique words in D
    V = set(word for document in D for word in document.split())

    # Convert class labels to NumPy array for vectorized operations
    C = np.array(C)
    unique_classes = np.unique(C)

    # Initialize log prior and likelihood dictionaries
    logprior = {}
    bigdoc = {c: [] for c in unique_classes}
    loglikelihood = {c: {} for c in unique_classes}

    # Calculate the number of documents
    N_doc = len(D)

    # Calculate logprior and fill bigdoc with words from all documents for each class
    for c in unique_classes:
        # Calculate the number of documents for each class
        N_c = np.sum(C == c)
        logprior[c] = np.log(N_c / N_doc)
        bigdoc[c] = np.hstack([document.split() for document, label in zip(D, C) if label == c])

        # Calculate the total count of words for each class with Laplace smoothing
        count_wc = np.array([(bigdoc[c] == word).sum() for word in V]) + 1
        total_count = count_wc.sum() + len(V)
        loglikelihood[c] = dict(zip(V, np.log(count_wc / total_count)))

    return logprior, loglikelihood, V , unique_classes


def test_naive_bayes(testdoc, logprior, loglikelihood, C, V):
    sum_c = {}
    for c in C:
        sum_c[c] = logprior[c]
        words = testdoc.split()
        for word in words:
            if word in V:
                sum_c[c] += loglikelihood[c].get(word, 0)
    return max(sum_c, key=sum_c.get)


In [ ]:
X_train = train_dataset['sentence']
y_train = train_dataset['label']
X_test = test_dataset['sentence']
y_test = test_dataset['label']

In [ ]:
logprior, loglikelihood, V, unique_classes = train_naive_bayes(X_train, y_train)

In [ ]:
def predict_labels(X_test, logprior, loglikelihood, unique_classes, V):
    y_pred = []
    for doc in X_test:
        predicted_label = test_naive_bayes(doc, logprior, loglikelihood, unique_classes, V)
        y_pred.append(predicted_label)
    return y_pred

def calculate_accuracy(Y_test, Y_pred):
    correct_predictions = np.sum(np.array(Y_test) == np.array(Y_pred))
    accuracy = correct_predictions / len(Y_test)
    return accuracy

In [ ]:
y_pred = predict_labels(X_test,logprior, loglikelihood, unique_classes, V)

In [ ]:
accuracy_custom = calculate_accuracy(y_test, y_pred)
print("Accuracy:", accuracy_custom)

Accuracy: 0.3909502262443439


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

# Create a pipeline that first transforms the data using CountVectorizer and then applies MultinomialNB
pipeline = make_pipeline(CountVectorizer(), MultinomialNB(alpha=1.0))

# Train the model
pipeline.fit(X_train, y_train)

# Predict the labels for the test set
y_pred_scikit = pipeline.predict(X_test)

# Calculate the accuracy
accuracy_scikit = calculate_accuracy(y_test, y_pred_scikit)
print("Accuracy:", accuracy_scikit)


Accuracy: 0.4076923076923077


In [ ]:
import numpy as np
from nltk.util import ngrams
from collections import defaultdict

# Tokenize each sentence into words
def tokenize_sentence(sentence):
    return sentence.split()

# Generate word bi-grams from a list of words
def generate_word_bigrams(words):
    return list(ngrams(words, 2))

# Create a vocabulary of unique word bi-grams from the entire dataset
def create_vocab(dataset):
    vocab = set()
    for example in dataset:
        words = tokenize_sentence(example['sentence'])
        word_bigrams = generate_word_bigrams(words)
        vocab.update(word_bigrams)
    return sorted(list(vocab))

# Convert each set of sentences into a feature vector
def sentences_to_features(sentences_list, vocab):
    features = []
    for sentence in sentences_list:
        words = tokenize_sentence(sentence)
        word_bigrams = generate_word_bigrams(words)
        vector = [1 if bi_gram in word_bigrams else 0 for bi_gram in vocab]
        features.append(vector)
    return np.array(features)

vocab = create_vocab(train_dataset)

In [ ]:
X_train = sentences_to_features([example['sentence'] for example in train_dataset], vocab)

In [ ]:
X_train.shape

(8544, 63042)

In [ ]:
X_valid = sentences_to_features([example['sentence'] for example in valid_dataset], vocab)

In [ ]:
X_test = sentences_to_features([example['sentence'] for example in test_dataset], vocab)

In [ ]:
print(X_train[50])

[0 0 0 ... 0 0 0]


In [ ]:
class MulticlassLogisticRegression:
    def __init__(self, num_features, num_classes):
        self.num_features = num_features
        self.num_classes = num_classes
        self.weights = np.zeros((num_features, num_classes))  # Initialize weights
        self.bias = np.zeros(num_classes)  # Initialize bias for each class

    def one_hot_encode(self, labels, num_classes):
        num_samples = len(labels)
        one_hot_labels = np.zeros((num_samples, num_classes), dtype=np.int8)
        for i, label in enumerate(labels):
            label = int(label)  # Ensure label is converted to an integer
            one_hot_labels[i, label] = 1
        return one_hot_labels


    def softmax(self, z):
        exp_z = np.exp(z)
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def forward(self, X):
        z = np.dot(X, self.weights) + self.bias
        return self.softmax(z)

    def compute_loss(self, y_pred, y_true):
        epsilon = 1e-15  # Small value to avoid numerical instability
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip values to avoid log(0)
        num_samples = y_true.shape[0]
        # Compute the cross-entropy loss for all samples in the batch
        loss = -np.mean(np.sum(y_true * np.log(y_pred), axis=1))
        return loss

    def backward(self, X, y_pred, y_true):
        m = len(y_true)
        dz = y_pred - y_true
        dw = np.dot(X.T, dz) / m  # Compute gradients for weights
        db = np.sum(dz, axis=0) / m  # Compute gradients for bias
        return dw, db

    def update_parameters(self, dw, db, learning_rate):
        self.weights -= learning_rate * dw
        self.bias -= learning_rate * db

    def train(self, X_train, y_train, num_epochs, batch_size, learning_rate):
        num_samples = X_train.shape[0]
        y_train_encoded = self.one_hot_encode(y_train, self.num_classes)  # One-hot encode y_train
        for epoch in range(num_epochs):
            # Shuffle the data
            indices = np.random.permutation(num_samples)
            X_train_shuffled = X_train[indices]
            y_train_shuffled = y_train_encoded[indices]

            # Mini-batch training
            for i in range(0, num_samples, batch_size):
                X_batch = X_train_shuffled[i:i+batch_size]
                y_batch = y_train_shuffled[i:i+batch_size]

                # Forward pass
                y_pred = self.forward(X_batch)

                # Compute loss
                loss = self.compute_loss(y_pred, y_batch)

                # Backward pass
                dw, db = self.backward(X_batch, y_pred, y_batch)

                # Update parameters
                self.update_parameters(dw, db, learning_rate)

            # Print loss at the end of each epoch
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}')

In [ ]:
def evaluate_model(model, X_valid, y_valid):
    # Predict probabilities for each class
    y_pred_proba = model.forward(X_valid)

    # Convert probabilities to class labels
    y_pred = np.argmax(y_pred_proba, axis=1)

    # Calculate accuracy
    accuracy = np.mean(y_pred == y_valid)

    return accuracy

In [ ]:
model = MulticlassLogisticRegression(num_features=X_train.shape[1], num_classes=5)

model.train(X_train, y_train, num_epochs=5, batch_size=32, learning_rate=0.01)

# Evaluate model performance on the validation set
accuracy = evaluate_model(model, X_valid, y_valid)
print("Validation Accuracy:", accuracy)

Epoch 1/5, Loss: 1.5721


In [ ]:
accuracy = evaluate_model(model, X_test, y_test)
print("Test Accuracy:", accuracy)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Train scikit-learn's logistic regression model
sklearn_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
sklearn_model.fit(X_train, y_train)

# Evaluate performance on validation set
sklearn_val_accuracy = sklearn_model.score(X_valid, y_valid)

# Evaluate performance on test set
sklearn_test_accuracy = sklearn_model.score(X_test, y_test)

print("Scikit-learn Implementation - Validation Accuracy:", sklearn_val_accuracy)
print("Scikit-learn Implementation - Test Accuracy:", sklearn_test_accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Scikit-learn Implementation - Validation Accuracy: 0.28973660308810173
Scikit-learn Implementation - Test Accuracy: 0.28868778280542984


In [ ]:
del sklearn_model

In [ ]:
def confusion_matrix(y_true, y_pred, labels):

    num_classes = len(labels)
    label_to_index = {label: index for index, label in enumerate(labels)}
    conf_matrix = np.zeros((num_classes, num_classes), dtype=int)

    for true, pred in zip(y_true, y_pred):
        true_index = label_to_index[true]
        pred_index = label_to_index[pred]
        conf_matrix[true_index, pred_index] += 1

    return conf_matrix

def compute_metrics(conf_matrix):

    precision = np.diag(conf_matrix) / np.sum(conf_matrix, axis=0)
    recall = np.diag(conf_matrix) / np.sum(conf_matrix, axis=1)
    f1 = 2 * (precision * recall) / (precision + recall)

    # Compute macro averages
    macro_precision = np.nanmean(precision)
    macro_recall = np.nanmean(recall)
    macro_f1 = np.nanmean(f1)

    return precision, recall, f1, macro_precision, macro_recall, macro_f1


In [ ]:
confusion_matrix_custom=confusion_matrix(y_test,y_pred,unique_classes)
precision, recall, f1, macro_precision, macro_recall, macro_f1=compute_metrics(confusion_matrix_custom)
print("Scores of naive bayes using custom model:")
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Macro Precision:", macro_precision)
print("Macro Recall:", macro_recall)
print("Macro F1 Score:", macro_f1)
print("\n")
confusion_matrix_scikit=confusion_matrix(y_test,y_pred_scikit,unique_classes)
precision, recall, f1, macro_precision, macro_recall, macro_f1=compute_metrics(confusion_matrix_scikit)
print("Scores of naive bayes using scikit model:")
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Macro Precision:", macro_precision)
print("Macro Recall:", macro_recall)
print("Macro F1 Score:", macro_f1)